In [32]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.autograd.variable import Variable
from Discriminator_hojun import Discriminator
from Generator_hojun import Generator, make_layers, get_activation
from utils.loaders import load_safari

import numpy as np
import os
import matplotlib.pyplot as plot
import json
import pickle as pkl
import matplotlib.pyplot as plt

In [40]:
num_epochs = 100
batch_size = 128
learning_rate = 0.0004
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
DATA_NAME='camel'
(x_train, y_train) = load_safari(DATA_NAME)
_x_train = np.transpose(x_train, (0,3,1,2))
_y_train=np.array(y_train)
x_train = torch.from_numpy(_x_train).to(device)
y_train= torch.from_numpy(_y_train).to(device)
input_data = [[x_item, y_item] for x_item, y_item in zip(x_train, y_train)]

In [41]:
discriminator = Discriminator(input_dim=(60,1,28,28), discriminator_conv_channels = [64,64,128,128],
    discriminator_conv_kernel_size = [5,5,5,5], discriminator_conv_strides = [2,2,2,1],
    discriminator_batch_norm_momentum = None, discriminator_activation = 'relu',
    discriminator_dropout_rate = 0.4).to(device)
generator=Generator(input_dim=100, generator_initial_dense_layer_size = (64, 7, 7)
        , generator_upsample = [2,2, 1, 1]
        , generator_conv_channels = [128,64, 64,1]
        , generator_conv_kernel_size = [5,5,5,5]
        , generator_conv_strides = [1,1, 1, 1]
        , generator_batch_norm_momentum = 0.9
        , generator_activation = 'relu').to(device)

In [42]:
def init_params(model):
    for p in model.parameters():
        if(p.dim() > 1):
            nn.init.xavier_normal_(p)
        else:
            nn.init.uniform_(p, 0.1, 0.2)

In [43]:
init_params(generator)
init_params(discriminator)

In [44]:
camel_loader=DataLoader(input_data, batch_size=batch_size, shuffle=True)
# Loss and optimizer
criterion = nn.BCELoss()

discriminator_optimizer = torch.optim.Adam(discriminator.parameters(), lr=learning_rate)
generator_optimizer = torch.optim.Adam(generator.parameters(), lr=learning_rate)


In [45]:
# Train the model
num_epochs=10
for epoch in range(num_epochs):
    for _, (x_batch, y_batch) in enumerate(camel_loader):
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        # Forward pass
        if len(x_batch)<batch_size:
            valid_discriminator = torch.ones((len(x_batch),1))
            fake_discriminator = torch.zeros((len(x_batch),1))
        else:
            valid_discriminator = torch.ones((batch_size,1))
            fake_discriminator = torch.zeros((batch_size,1))
        
        valid_discriminator=Variable(valid_discriminator).to(device)
        fake_discriminator=Variable(fake_discriminator).to(device)
        noise =torch.normal(0, 1.0, (batch_size, 100))
        noise = Variable(torch.Tensor(noise)).to(device)
        gen_x_batch = generator(noise)
        total_x_batch=torch.cat((gen_x_batch, x_batch), 0)
        predicted_labels=discriminator(total_x_batch)
        labels=torch.cat((fake_discriminator, valid_discriminator), 0)
        predicted_labels=predicted_labels.squeeze(1)
        labels=labels.squeeze(1)
        
        discriminator_loss = criterion(predicted_labels, labels).to(device)
        # Backward and optimize
        discriminator_optimizer.zero_grad()
        discriminator_loss.backward()
        discriminator_optimizer.step()
        
        
        valid_generator = Variable(torch.ones([batch_size,1])).to(device).squeeze(1)
        noise_generator = Variable(torch.Tensor(np.random.normal(0, 1, (batch_size, 100)))).to(device)
        output_generator = generator(noise_generator)
        output_generator = discriminator(output_generator).squeeze(1)
        generator_loss=criterion(output_generator, valid_generator)
        generator_optimizer.zero_grad()
        generator_loss.backward()
        generator_optimizer.step()
        
    print("%d: Discriminator Loss=%.4f, Generator Loss=%.4f" % (
        epoch, 10000*discriminator_loss.item(), generator_loss.item()))
        

0: Discriminator Loss=1.0829, Generator Loss=0.2703
1: Discriminator Loss=0.3433, Generator Loss=0.3403
2: Discriminator Loss=0.8593, Generator Loss=1.2696
3: Discriminator Loss=0.1432, Generator Loss=1.2632
4: Discriminator Loss=0.1375, Generator Loss=1.7921
5: Discriminator Loss=0.1485, Generator Loss=2.3289
6: Discriminator Loss=0.4046, Generator Loss=1.8123
7: Discriminator Loss=0.0288, Generator Loss=1.5002
8: Discriminator Loss=0.0028, Generator Loss=3.0021
9: Discriminator Loss=0.0137, Generator Loss=2.1478


In [56]:
test_sample=torch.normal(0, 1, (1, 100))
test_sample=test_sample.to(device)
test_sample.shape
test_result=generator(test_sample)


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 3136])

In [57]:
print(generator)

Generator(
  (generator_layer1): Sequential(
    (0): Linear(in_features=100, out_features=3136, bias=True)
    (1): BatchNorm1d(3136, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (generator_layer2): Sequential(
    (0): Upsample(scale_factor=2.0, mode=nearest)
    (1): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (2): BatchNorm2d(128, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (3): ReLU()
  )
  (generator_layer3): Sequential(
    (0): Upsample(scale_factor=2.0, mode=nearest)
    (1): Conv2d(128, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (3): ReLU()
  )
  (generator_layer4): Sequential(
    (0): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (generator_layer5): Se

In [6]:
# test_x=torch.Tensor(np.random.normal(0, 1, (batch_size, 100)))
# class generator_layer1(nn.Module):
#     def __init__(self):
#         super(generator_layer1, self).__init__()
#         self.conv=nn.Sequential(
#             nn.Linear(100, 3136)
#         )
#     def forward(self, x):
#         x=self.conv(x)
#         return(x)
# generator(test_x)

In [7]:
# class generator_layer2(nn.Module):
#     def __init__(self, in_channel=64, out_channel=128, kernel_size=5, padding=2, batchnorm_momentum=0.1):
#         super(generator_layer2, self).__init__()
#        self.conv=nn.Sequential(
#        nn.Upsample(scale_factor=2),
#        nn.Conv2d(in_channel, out_channel, kernel_size=kernel_size, padding=padding),
#        nn.BatchNorm2d(out_channel, momentum=batchnorm_momentum),
#        get_activation()
#        )
#         self.conv1=nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, padding=2)
        
#     make_layers(in_channel=64, out_channel=128, kernel_size=5, padding=2, batchnorm_momentum=0.1)
#     def forward(self, x):
#         x=x.view(-1, 64, 7, 7)
#         print(x.shape)
#         x=self.conv1(x)
#         return(x)